# Base Lucro Bruto + View

Esta base faz merge dos dados do Lucro Bruto com os dados da View v18


Esta base extrai 3 diferentes niveis de granulariedade e as colunas presentes dependem deste nivel

Nivel: Data Fatura + Oridem + Item [Chave: Ordem + Item]
Nivel: Data Fatura + SKU [Chave: SKU]
Nivel: Data Fatura + GPD [Chave: GPD]

O código desenvolvido em PySpark

Utilizando boto3 para fazer query direto via Athena para dados da View
E carregando .parquet do LucroBruto


## Imports & Configs

In [36]:
#imports Libs

import os
import pandas as pd
import boto3
import time
from botocore.client import ClientError

import pyarrow.parquet as pq
import s3fs

import findspark                                              #Import library to Search for Spark Installation  

findspark.init()                                              #Search Spark Installation

import pyspark                                                #Only run after findspark.init()

from pyspark.sql import SparkSession                          #Import of Spark Session
from pyspark import SparkContext as spark                     #Import the Regular Spark Contex 
from pyspark.sql import SQLContext                            #Import the SQL Spark Contex 
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import year, month, dayofmonth
spark = SparkSession.builder.getOrCreate()



sc = spark.sparkContext                                       #Initialize Spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'pandas'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'pandas'



-------------

# Creating Variables

## Database

In [ ]:
ATHENA_SPG = os.environ['SPG_DATABASE']

In [ ]:
ATHENA_HANA = os.environ['GERDAU_HANA_DATABASE']

In [ ]:
ATHENA_SALES = os.environ['GERDAU_SALES_DATABASE']

## Buckets

In [ ]:
SPG_MANUAL_INPUT_BUCKET = os.environ['MANUAL_INPUT_BUCKET']

In [ ]:
SPG_INTEGRATION_INPUT_BUCKET = os.environ['INTEGRATION_INPUT_BUCKET']

In [ ]:
SPG_QUERY_BUCKET = os.environ['QUERY_BUCKET']

In [ ]:
GERDAU_BUCKET = os.environ['GERDAU_BUCKET']

In [ ]:
SPG_OUTPUT_BUCKET = os.environ['OUTPUT_BUCKET']

In [ ]:
SPG_INPUT_BUCKET = os.environ['INPUT_BUCKET_FROM_OUTPUT']

## Input Paths

In [ ]:
QUERY_VIEW = "SELECT billing_date, sales_order_date, sales_org_cod,gpm, gpd_cod, gpd_desc, material_cod, material_desc, sales_number, sales_item, issuing_city, issuing_state, quantity_ton, practiced_price, norm_practiced_price, politic_price, list_price, rbv, rlv, receiving_customer_cod, branch  FROM db_smart_pricing.vw_strategy_sample_v18  WHERE (billing_date >= '2019-01-01' or sales_order_date >= '2019-01-01')  AND manufacture LIKE '%Praticado%';" 

In [ ]:
SPG_INPUT_BUCKET_GP = "SPG_LB/LB_FULL"

## Output Paths

In [ ]:
SPG_OUTPUT_BUCKET_GP_STRATEGIC = "SPG_LB/LB_STRATEGIC"

In [ ]:
SPG_OUTPUT_BUCKET_GP_ORDER_ITEM = "SPG_LB/LB_OrdemItem"

In [ ]:
SPG_OUTPUT_BUCKET_GP_SALES_DATE = "SPG_LB/LB_SalesOrderDate"

In [ ]:
SPG_OUTPUT_BUCKET_GP_SKU = "SPG_LB/LB_SKU"

In [ ]:
SPG_OUTPUT_BUCKET_GP_GPD = "SPG_LB/LB_GPD"

In [ ]:
SPG_OUTPUT_BUCKET_GP_OV = "SPG_LB/LB_OV"

In [ ]:
SPG_OUTPUT_BUCKET_GP_SOFC = "SPG_LB/LB_SOFC"

In [ ]:
SPG_OUTPUT_BUCKET_GP_SF = "SPG_LB/LB_SF.csv"

## Boto3 Variables

In [ ]:
#S3 Configuration
S3_ATHENA_INPUT =  's3://'+SPG_QUERY_BUCKET+'/'+SPG_QUERY_BUCKET_ATHENA

In [ ]:
S3_ATHENA_OUTPUT = 's3://'+SPG_QUERY_BUCKET+'/'+SPG_QUERY_BUCKET_ATHENA

In [ ]:
region_name = os.environ['AWS_REGION']

In [ ]:
aws_access_key_id = os.environ['AWS_ACCESS_KEY']

In [ ]:
aws_secret_access_key = os.environ['AWS_SECRET_KEY']

-------------

# Creating Defined Functions

In [ ]:
# Run Query

def run_query(query, database, s3_output):
    response = client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
            },
        ResultConfiguration={
            'OutputLocation': s3_output,
            }
        )
    return response

In [ ]:
def get_aws_path(query,database,s3_output):
    response = run_query(query, database, s3_output)
    file_query = response['QueryExecutionId']
    file_metadata = response['QueryExecutionId'] + '.metadata'
    return file_query

In [ ]:
# Wating for 300 seconds until the end of the upload

def wait_athena_load(Bucket, Key):
    time_to_wait = 300
    time_counter = 0

    while True:
        try:
            s3.meta.client.head_object(Bucket=Bucket,Key=Key)
        except ClientError:
            time.sleep(1)
            time_counter += 1
            if time_counter > time_to_wait:
                break
        else:
            break

-------------

# Configuring Boto3

In [ ]:
#Athena Client Configuration

client = boto3.client('athena', 
    aws_access_key_id = aws_access_key_id, 
    aws_secret_access_key = aws_secret_access_key, 
    region_name = region_name )

In [ ]:
#S3 Resource Configuration

s3 = boto3.resource('s3',
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key,
    region_name = region_name)

----------

# Importing Tables

In [11]:
# import Lucro Bruto

df_lb_full = spark.read.parquet("s3a://"+SPG_INPUT_BUCKET+"/"+SPG_INPUT_BUCKET_GP)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Import CSV from View

athena_response = get_aws_path(QUERY_VIEW,ATHENA_SPG,S3_ATHENA_OUTPUT)

wait_athena_load(SPG_QUERY_BUCKET, SPG_QUERY_BUCKET_ATHENA+"/"+athena_response+".csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'database' is not defined
Traceback (most recent call last):
NameError: name 'database' is not defined



In [10]:
df_view = spark.read.csv("s3a://"+SPG_QUERY_BUCKET+"/"+SPG_QUERY_BUCKET_ATHENA+"/"+athena_response+".csv", header = 'true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'view_path' is not defined
Traceback (most recent call last):
NameError: name 'view_path' is not defined



---------

# Preparing Table

In [12]:
# BKP_DF 

ordem_view =  df_view

df_lb = df_lb_full.drop("GTC101018", 'GTC100511', 'GTC100020')

df_lb = df_lb.withColumn("QUANTITY_TON_BW", col("QUANTITY_TON").cast("float"))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'df_view' is not defined
Traceback (most recent call last):
NameError: name 'df_view' is not defined



In [13]:
df_lb_full = df_lb_full \
    .withColumnRenamed("GTC101018","ISSUING_STATE")\
    .withColumnRenamed("DATA","SALES_ORDER_DATE")\
    .withColumnRenamed("SALES_NUMBER_ITEM","SALES_ITEM")\
    .withColumnRenamed("GTC100511","SALES_ORG_COD")\
    .withColumnRenamed("GTC100020","MATERIAL_COD")\
    .withColumn('year', year(col('SALES_ORDER_DATE'))) \
    .withColumn('month', month(col('SALES_ORDER_DATE')))\
    .withColumn('KEY_LB', concat(col('SALES_NUMBER'),lit('_'),col('SALES_ITEM')))\
    .withColumn('UF', concat(lit('BR/'),col('ISSUING_STATE')))\
    .withColumn('FREIGHT_Full', col('SHIP_DEL_LOAD_C_C') + col('PORT_EXPENSES') + col('FREIGHT') + col('COGS_ADJUSTMENTS'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'year' is not defined
Traceback (most recent call last):
NameError: name 'year' is not defined



In [14]:
df_lb_full = df_lb_full\
    .withColumn("SALES_ITEM", col("SALES_ITEM").cast("int"))\
    .withColumn("QUANTITY_TON_BW", col("QUANTITY_TON").cast("float"))\
    .withColumn("SALES_NUMBER", col("SALES_NUMBER").cast("int"))\
    .withColumn("GPD_COD", col("GPD_COD").cast("int"))\
    .withColumn("MATERIAL_COD", col("MATERIAL_COD").cast("int"))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'col' is not defined
Traceback (most recent call last):
NameError: name 'col' is not defined



In [15]:
filtred_OV = ['BRIN','BRIO','BRDI','BRDO','BRCC','BRCO','BRCG','BRGO']

df_lb_full = df_lb_full.where(df_lb_full.SALES_ORG_COD.isin(filtred_OV))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'DataFrame' object has no attribute 'SALES_ORG_COD'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 1301, in __getattr__
    "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
AttributeError: 'DataFrame' object has no attribute 'SALES_ORG_COD'



In [16]:
df_lb_full=df_lb_full.withColumn("SALES_ORG_COD", when(df_lb_full.SALES_ORG_COD.like("BRCO"), "BRCC")\
                                                    .otherwise(when(df_lb_full.SALES_ORG_COD.like("BRGO"), "BRCG")\
                                                              .otherwise(when(df_lb_full.SALES_ORG_COD.like("BRIO"), "BRIN")\
                                                                        .otherwise(when(df_lb_full.SALES_ORG_COD.like("BRDO"), "BRDI")
                                                                                  .otherwise(df_lb_full.SALES_ORG_COD)))))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'when' is not defined
Traceback (most recent call last):
NameError: name 'when' is not defined



In [17]:
df_lb_full=df_lb_full.join(df_view.select(df_view.material_cod.cast("int").alias("MATERIAL_COD")
                                          ,df_view.sales_org_cod.alias("SALES_ORG_COD")
                                          ,df_view.gpd_cod.cast("int").alias("gpd_cod_drop")
                                          ,df_view.gpd_desc.alias("GPD_DESC")).distinct(),on=['material_cod','sales_org_cod'],how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'df_view' is not defined
Traceback (most recent call last):
NameError: name 'df_view' is not defined



In [18]:
df_lb_full=df_lb_full.withColumn("GPD_COD", when(df_lb_full.GPD_COD.isNull(), df_lb_full.gpd_cod_drop)\
                                                .otherwise(df_lb_full.GPD_COD))\
                     .drop("gpd_cod_drop")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'when' is not defined
Traceback (most recent call last):
NameError: name 'when' is not defined



In [19]:
#df_lb_full=df_lb_full.filter(~df_lb_full.gpd_desc.isNull())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Ordem + Item

In [20]:
# Cria Coluna Mês e Ano

ordem_view = ordem_view \
            .withColumn('year', year(col('billing_date'))) \
            .withColumn('month', month(col('billing_date')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'ordem_view' is not defined
Traceback (most recent call last):
NameError: name 'ordem_view' is not defined



In [21]:
# Group BY ordem_item por data

ordem_item = ordem_view \
    .withColumn("pp_x_volume", col("practiced_price") * col("quantity_ton")) \
    .withColumn("ppn_x_volume", col("preco_prat_norm") * col("quantity_ton")) \
    .groupBy('year', 'month', 'sales_order_date','gpm','gpd_cod','gpd_desc', 'material_desc', 'material_cod' ,'sales_number','sales_item','sales_org_cod','receiving_customer_cod','branch','issuing_state') \
    .agg(F.sum("quantity_ton").alias("quantity_ton"), \
         F.sum("rbv").alias("rbv"), \
         F.sum("rlv").alias("rlv"), \
         F.sum("practiced_price").alias("sum_practiced_price"), \
         F.sum("pp_x_volume").alias("sum_pp_x_volume"), \
         F.sum("preco_prat_norm").alias("sum_preco_prat_norm"), \
         F.sum("ppn_x_volume").alias("sum_ppn_x_volume")) \
    .withColumn("practiced_price_weighted_average", col("sum_practiced_price") / col("sum_pp_x_volume")) \
    .withColumn("preco_prat_norm_weighted_average", col("sum_preco_prat_norm") / col("sum_ppn_x_volume")) \
    .drop('pp_x_volume', 'ppn_x_volume', 'sum_practiced_price','sum_pp_x_volume','sum_preco_prat_norm','sum_ppn_x_volume')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'ordem_view' is not defined
Traceback (most recent call last):
NameError: name 'ordem_view' is not defined



## LB_View Ordem + Item

In [22]:
# Cria chave para Join

ordem_item = ordem_item \
    .withColumn('KEY_LB_DATA', concat(col('year'), lit('-'), col('month'), lit('_'),col('sales_number'), lit('_'), col('sales_item')))

# Sumariza Frete

df_lb = df_lb \
    .withColumn('FREIGHT_Full', col('SHIP_DEL_LOAD_C_C') + col('PORT_EXPENSES') + col('FREIGHT') + col('COGS_ADJUSTMENTS'))
                
  
# Select Colunas LB

df_lb = df_lb.select('KEY_LB_DATA', 'COGS_TOTAL','GROSS_PROFIT_BW','NET_SALES','FREIGHT_Full','Quantity_ton_bw', 'COMISSION_TO_AGENTS')


# Join Ordem View + Lucro Bruto

LB_View = ordem_item.join(df_lb, on=['KEY_LB_DATA'], how='left')

# Limpa nulls

LB_View = LB_View.filter(LB_View.GROSS_PROFIT_BW.isNotNull())

# Cria chave para View por Fatura

LB_View = LB_View \
    .withColumn('KEY_LB', concat(col('sales_number'), lit('_'), col('sales_item')))



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'ordem_item' is not defined
Traceback (most recent call last):
NameError: name 'ordem_item' is not defined



## SALES ORDER DATE

In [23]:
sales_order_date = LB_View \
    .withColumn("pp_x_volume", col("practiced_price_weighted_average") * col("quantity_ton")) \
    .withColumn("ppn_x_volume", col("preco_prat_norm_weighted_average") * col("quantity_ton")) \
    .groupBy('KEY_LB','sales_order_date','gpm','gpd_cod','gpd_desc', 'material_desc', 'material_cod' ,'sales_number','sales_item') \
    .agg(F.sum("quantity_ton").alias("quantity_ton"), \
         F.sum("rbv").alias("rbv"), \
         F.sum("rlv").alias("rlv"), \
         F.sum("practiced_price_weighted_average").alias("sum_practiced_price"), \
         F.sum("pp_x_volume").alias("sum_pp_x_volume"), \
         F.sum("preco_prat_norm_weighted_average").alias("sum_preco_prat_norm"), \
         F.sum("ppn_x_volume").alias("sum_ppn_x_volume"), \
         F.sum("COGS_TOTAL").alias("COGS_TOTAL"), \
         F.sum("Quantity_ton_bw").alias("Quantity_ton_bw"), \
         F.sum("FREIGHT_Full").alias("FREIGHT_Full"), \
         F.sum("GROSS_PROFIT_BW").alias("GROSS_PROFIT_BW"), \
         F.sum("NET_SALES").alias("NET_SALES")) \
    .withColumn("practiced_price_weighted_average", col("sum_practiced_price") / col("sum_pp_x_volume")) \
    .withColumn("preco_prat_norm_weighted_average", col("sum_preco_prat_norm") / col("sum_ppn_x_volume")) \
    .drop('pp_x_volume', 'ppn_x_volume', 'sum_practiced_price','sum_pp_x_volume','sum_preco_prat_norm','sum_ppn_x_volume')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'LB_View' is not defined
Traceback (most recent call last):
NameError: name 'LB_View' is not defined



## SKU OV FILIAL CLIENTE

In [24]:
# Group by SKU

sku_ov_filial_cliente = LB_View \
    .withColumn("pp_x_volume", col("practiced_price_weighted_average") * col("quantity_ton")) \
    .withColumn("ppn_x_volume", col("preco_prat_norm_weighted_average") * col("quantity_ton")) \
    .groupBy('year', 'month','gpm','gpd_cod','gpd_desc', 'material_desc', 'material_cod','receiving_customer_cod','branch','sales_org_cod') \
    .agg(F.sum("quantity_ton").alias("quantity_ton"), \
         F.sum("Quantity_ton_bw").alias("Quantity_ton_bw"), \
         F.sum("rbv").alias("rbv"), \
         F.sum("rlv").alias("rlv"), \
         F.sum("practiced_price_weighted_average").alias("sum_practiced_price"), \
         F.sum("pp_x_volume").alias("sum_pp_x_volume"), \
         F.sum("preco_prat_norm_weighted_average").alias("sum_preco_prat_norm"), \
         F.sum("ppn_x_volume").alias("sum_ppn_x_volume"), \
         F.sum("COGS_TOTAL").alias("COGS_TOTAL"), \
         F.sum("FREIGHT_Full").alias("FREIGHT_Full"), \
         F.sum("GROSS_PROFIT_BW").alias("GROSS_PROFIT_BW"), \
         F.sum("NET_SALES").alias("NET_SALES")) \
    .withColumn("practiced_price_weighted_average", col("sum_practiced_price") / col("sum_pp_x_volume")) \
    .withColumn("preco_prat_norm_weighted_average", col("sum_preco_prat_norm") / col("sum_ppn_x_volume")) \
    .drop('pp_x_volume', 'ppn_x_volume', 'sum_practiced_price','sum_pp_x_volume','sum_preco_prat_norm','sum_ppn_x_volume')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'LB_View' is not defined
Traceback (most recent call last):
NameError: name 'LB_View' is not defined



## SKU

In [25]:
# Group by SKU/OV

sku = LB_View \
    .withColumn("pp_x_volume", col("practiced_price_weighted_average") * col("quantity_ton")) \
    .withColumn("ppn_x_volume", col("preco_prat_norm_weighted_average") * col("quantity_ton")) \
    .groupBy('year', 'month','gpm','gpd_cod','gpd_desc', 'material_desc', 'material_cod','sales_org_cod') \
    .agg(F.sum("quantity_ton").alias("quantity_ton"), \
         F.sum("Quantity_ton_bw").alias("Quantity_ton_bw"), \
         F.sum("rbv").alias("rbv"), \
         F.sum("rlv").alias("rlv"), \
         F.sum("practiced_price_weighted_average").alias("sum_practiced_price"), \
         F.sum("pp_x_volume").alias("sum_pp_x_volume"), \
         F.sum("preco_prat_norm_weighted_average").alias("sum_preco_prat_norm"), \
         F.sum("ppn_x_volume").alias("sum_ppn_x_volume"), \
         F.sum("COGS_TOTAL").alias("COGS_TOTAL"), \
         F.sum("FREIGHT_Full").alias("FREIGHT_Full"), \
         F.sum("GROSS_PROFIT_BW").alias("GROSS_PROFIT_BW"), \
         F.sum("NET_SALES").alias("NET_SALES")) \
    .withColumn("practiced_price_weighted_average", col("sum_practiced_price") / col("sum_pp_x_volume")) \
    .withColumn("preco_prat_norm_weighted_average", col("sum_preco_prat_norm") / col("sum_ppn_x_volume")) \
    .drop('pp_x_volume', 'ppn_x_volume', 'sum_practiced_price','sum_pp_x_volume','sum_preco_prat_norm','sum_ppn_x_volume')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'LB_View' is not defined
Traceback (most recent call last):
NameError: name 'LB_View' is not defined



## OV

In [26]:
# Group by OV

sales_order = LB_View \
    .withColumn("pp_x_volume", col("practiced_price_weighted_average") * col("quantity_ton")) \
    .withColumn("ppn_x_volume", col("preco_prat_norm_weighted_average") * col("quantity_ton")) \
    .groupBy('year', 'month','gpm','gpd_cod','gpd_desc','sales_org_cod') \
    .agg(F.sum("quantity_ton").alias("quantity_ton"), \
         F.sum("Quantity_ton_bw").alias("Quantity_ton_bw"), \
         F.sum("rbv").alias("rbv"), \
         F.sum("rlv").alias("rlv"), \
         F.sum("practiced_price_weighted_average").alias("sum_practiced_price"), \
         F.sum("pp_x_volume").alias("sum_pp_x_volume"), \
         F.sum("preco_prat_norm_weighted_average").alias("sum_preco_prat_norm"), \
         F.sum("ppn_x_volume").alias("sum_ppn_x_volume"), \
         F.sum("COGS_TOTAL").alias("COGS_TOTAL"), \
         F.sum("FREIGHT_Full").alias("FREIGHT_Full"), \
         F.sum("GROSS_PROFIT_BW").alias("GROSS_PROFIT_BW"), \
         F.sum("NET_SALES").alias("NET_SALES")) \
    .withColumn("practiced_price_weighted_average", col("sum_practiced_price") / col("sum_pp_x_volume")) \
    .withColumn("preco_prat_norm_weighted_average", col("sum_preco_prat_norm") / col("sum_ppn_x_volume")) \
    .drop('pp_x_volume', 'ppn_x_volume', 'sum_practiced_price','sum_pp_x_volume','sum_preco_prat_norm','sum_ppn_x_volume')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'LB_View' is not defined
Traceback (most recent call last):
NameError: name 'LB_View' is not defined



## GPD

In [27]:
# Group by GPD

gpd = LB_View \
    .withColumn("pp_x_volume", col("practiced_price_weighted_average") * col("quantity_ton")) \
    .withColumn("ppn_x_volume", col("preco_prat_norm_weighted_average") * col("quantity_ton")) \
    .groupBy('year', 'month','gpm','gpd_cod','gpd_desc') \
    .agg(F.sum("quantity_ton").alias("quantity_ton"), \
         F.sum("Quantity_ton_bw").alias("Quantity_ton_bw"), \
         F.sum("rbv").alias("rbv"), \
         F.sum("rlv").alias("rlv"), \
         F.sum("practiced_price_weighted_average").alias("sum_practiced_price"), \
         F.sum("pp_x_volume").alias("sum_pp_x_volume"), \
         F.sum("preco_prat_norm_weighted_average").alias("sum_preco_prat_norm"), \
         F.sum("ppn_x_volume").alias("sum_ppn_x_volume"), \
         F.sum("COGS_TOTAL").alias("COGS_TOTAL"), \
         F.sum("FREIGHT_Full").alias("FREIGHT_Full"), \
         F.sum("GROSS_PROFIT_BW").alias("GROSS_PROFIT_BW"), \
         F.sum("NET_SALES").alias("NET_SALES")) \
    .withColumn("practiced_price_weighted_average", col("sum_practiced_price") / col("sum_pp_x_volume")) \
    .withColumn("preco_prat_norm_weighted_average", col("sum_preco_prat_norm") / col("sum_ppn_x_volume")) \
    .drop('pp_x_volume', 'ppn_x_volume', 'sum_practiced_price','sum_pp_x_volume','sum_preco_prat_norm','sum_ppn_x_volume')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'LB_View' is not defined
Traceback (most recent call last):
NameError: name 'LB_View' is not defined



## SALES FORCE - SKU ESTADO OV

In [28]:

# Group by MES SKU ESTADO OV

view_sf = LB_View \
    .groupBy('month','gpm','gpd_cod','gpd_desc', 'material_desc', 'material_cod','sales_org_cod','issuing_state') \
    .agg(F.sum("rlv").alias("rlv"), \
         F.sum("COGS_TOTAL").alias("COGS_TOTAL"), \
         F.sum("GROSS_PROFIT_BW").alias("GROSS_PROFIT_BW"))    



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'LB_View' is not defined
Traceback (most recent call last):
NameError: name 'LB_View' is not defined



In [29]:
# cria chave


view_sf = view_sf \
    .withColumn('KEY_SF', concat(col('material_cod'), col('material_cod'), lit('_'), col('sales_org_cod'), lit('_'),col('issuing_state')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'view_sf' is not defined
Traceback (most recent call last):
NameError: name 'view_sf' is not defined



In [30]:
# Filtra meses

thiyear = [1, 2, 3, 4, 5, 7, 8, 9]

view_sf = view_sf[view_sf.month.isin(thiyear)]

lastmonth = [7, 8, 9]

df_sf = view_sf[view_sf.month.isin(lastmonth)]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'view_sf' is not defined
Traceback (most recent call last):
NameError: name 'view_sf' is not defined



In [31]:
# Cria as médias 3 

grouped = df_sf \
    .groupBy('KEY_SF' ) \
    .agg(F.sum("GROSS_PROFIT_BW").alias("GROSS_PROFIT_BWm"), \
        F.sum("COGS_TOTAL").alias("COGS_TOTALm"), \
        F.sum("RLV").alias("RLVm")) \
    .withColumn("avg_gross_profit_last_3", col("GROSS_PROFIT_BWm") / 3) \
    .withColumn("avg_cogs_last_3", col("COGS_TOTALm") / 3) \
    .withColumn("avg_rlv_last_3", col("RLVm") / 9) \
    .drop('GROSS_PROFIT_BWm', 'COGS_TOTALm', 'RLVm')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'df_sf' is not defined
Traceback (most recent call last):
NameError: name 'df_sf' is not defined



In [32]:
# Cria as médias ano
grouped12 = view_sf \
    .groupBy('KEY_SF' ) \
    .agg(F.sum("GROSS_PROFIT_BW").alias("GROSS_PROFIT_BWy"), \
    F.sum("COGS_TOTAL").alias("COGS_TOTALy"),
    F.sum("RLV").alias("RLVy")) \
    .withColumn("avg_gross_profit_year", col("GROSS_PROFIT_BWy") / 9) \
    .withColumn("avg_cogs_last_year", col("COGS_TOTALy") / 9) \
    .withColumn("avg_rlv_last_year", col("RLVy") / 9) \
    .drop('GROSS_PROFIT_BWy', 'COGS_TOTALy', 'RLVy')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'view_sf' is not defined
Traceback (most recent call last):
NameError: name 'view_sf' is not defined



In [33]:
# join 3 meses
view_sf = view_sf.join(grouped, on=['KEY_SF'], how='left') 

# join 12 meses

view_sf = view_sf.join(grouped12, on=['KEY_SF'], how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'view_sf' is not defined
Traceback (most recent call last):
NameError: name 'view_sf' is not defined



In [34]:
# Retira Mes

sf_sf = view_sf \
    .groupBy('KEY_SF', 'gpm','gpd_cod','gpd_desc', 'material_desc', 'material_cod','sales_org_cod','issuing_state') \
    .agg(F.sum("rlv").alias("rlv"), \
         F.sum("COGS_TOTAL").alias("COGS_TOTAL"), \
         F.sum("GROSS_PROFIT_BW").alias("GROSS_PROFIT_BW"), \
         F.avg("avg_gross_profit_last_3").alias("avg_gross_profit_last_3"), \
         F.avg("avg_cogs_last_3").alias("avg_cogs_last_3"), \
         F.avg("avg_rlv_last_3").alias("avg_rlv_last_3"), \
         F.avg("avg_gross_profit_year").alias("avg_gross_profit_year"), \
         F.avg("avg_cogs_last_year").alias("avg_cogs_last_year"), \
         F.avg("avg_rlv_last_year").alias("avg_rlv_last_year"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'view_sf' is not defined
Traceback (most recent call last):
NameError: name 'view_sf' is not defined



--------------

# Renaming Columns

In [35]:
for col_name in df_lb_full.columns:
    df_lb_full = df_lb_full.withColumnRenamed(col_name, col_name.upper())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'str' object has no attribute 'columns'
Traceback (most recent call last):
AttributeError: 'str' object has no attribute 'columns'



In [ ]:
for col_name in LB_View.columns:
    LB_View = LB_View.withColumnRenamed(col_name, col_name.upper())

In [ ]:
for col_name in sales_order_date.columns:
    sales_order_date = sales_order_date.withColumnRenamed(col_name, col_name.upper())

In [ ]:
for col_name in sku.columns:
    sku = sku.withColumnRenamed(col_name, col_name.upper())

In [ ]:
for col_name in gpd.columns:
    gpd = gpd.withColumnRenamed(col_name, col_name.upper())

In [ ]:
for col_name in sales_order.columns:
    sales_order = sales_order.withColumnRenamed(col_name, col_name.upper())

In [ ]:
for col_name in sku_ov_filial_cliente.columns:
    sku_ov_filial_cliente = sku_ov_filial_cliente.withColumnRenamed(col_name, col_name.upper())

In [ ]:
for col_name in sf_sf.columns:
    sf_sf = sf_sf.withColumnRenamed(col_name, col_name.upper())

------------

## Exports

In [33]:
df_lb_full.write.parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_GP_STRATEGIC, mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
# Export Ordem + Item

LB_View.write.parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_GP_ORDER_ITEM, mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# Export Sales Order Date

sales_order_date.write.parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_GP_SALES_DATE, mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# Export SKU

sku.write.parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_GP_SKU, mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
# Export GPD

gpd.write.parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_GP_GPD, mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
# Export OV

sales_order.write.parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_GP_OV, mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
# Export sku_ov_filial_cliente

sku_ov_filial_cliente.write.parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_GP_SOFC, mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
# Export Sales Force

sf_sf.repartition(1).write.option("header", "true").option("sep", ";")\
     .option("encoding", "Unicode").csv("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_GP_SF, mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…